<a href="https://colab.research.google.com/github/paulorfbr/transformers_course/blob/article_spinner/ArticleSpinner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Article spinner (masked language model)**

In [1]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2024-02-18 21:36:14--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  19.7MB/s    in 0.2s    

2024-02-18 21:36:14 (19.7 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [2]:
!pip install transformers

In [3]:
import numpy as np
import pandas as pd
import textwrap
from pprint import pprint

from transformers import pipeline

In [4]:
df = pd.read_csv('bbc_text_cls.csv')

In [5]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [6]:
labels = set(df['labels'])
labels

{'business', 'entertainment', 'politics', 'sport', 'tech'}

In [7]:
# Pick a label
label = 'business'

In [8]:
texts = df[df['labels'] == label]['text']
texts.head()

0    Ad sales boost Time Warner profit\n\nQuarterly...
1    Dollar gains on Greenspan speech\n\nThe dollar...
2    Yukos unit buyer faces loan claim\n\nThe owner...
3    High fuel prices hit BA's profits\n\nBritish A...
4    Pernod takeover talk lifts Domecq\n\nShares in...
Name: text, dtype: object

In [18]:
np.random.seed(1312)

In [19]:
i = np.random.choice(texts.shape[0])
doc = texts.iloc[i]

In [20]:
print(textwrap.fill(doc, replace_whitespace=False, fix_sentence_endings=True))

India's Deccan seals $1.8bn deal

Air Deccan has ordered 30 Airbus
A320 planes in a $1.8bn (£931m) deal as India's first low-cost airline
expands in the fast-growing domestic market.

Air Deccan was set up
last year and wants to lure travellers away from the railway network
and pricier rivals.  The potential of the Indian market has attracted
attention at home and abroad.  Beer magnate Vijay Mallya recently set
up Kingfisher Airlines, while UK entrepreneur Richard Branson has said
he is keen to start a local operation.

The country has a population
of more than a billion people and many observers feel that it is
underserved by airlines.  Recently however, the booming economy has
boosted personal spending power and helped swell the middle classes
and the corporate sector.  India's government has given its backing to
cheaper and more accessible air travel.  "The days of flying being a
symbol of only maharajas or the rich are over," the minister for civil
aviation Praful Patel said earlie

In [16]:
mlm = pipeline('fill-mask')

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [22]:
mlm(' The potential of the Indian market has attracted <mask>')

[{'score': 0.538027822971344,
  'token': 867,
  'token_str': ' investors',
  'sequence': ' The potential of the Indian market has attracted investors'},
 {'score': 0.044845253229141235,
  'token': 8666,
  'token_str': ' entrepreneurs',
  'sequence': ' The potential of the Indian market has attracted entrepreneurs'},
 {'score': 0.032160475850105286,
  'token': 6739,
  'token_str': ' Indians',
  'sequence': ' The potential of the Indian market has attracted Indians'},
 {'score': 0.02801680378615856,
  'token': 12784,
  'token_str': ' startups',
  'sequence': ' The potential of the Indian market has attracted startups'},
 {'score': 0.022623490542173386,
  'token': 915,
  'token_str': ' investment',
  'sequence': ' The potential of the Indian market has attracted investment'}]

In [25]:
text = 'Recently however, the <mask> economy has boosted personal spending power and helped swell the middle classes and the corporate sector.  India''s government has given its backing to cheaper and more accessible air travel.'

pprint(mlm(text))

[{'score': 0.1760990172624588,
  'sequence': 'Recently however, the booming economy has boosted personal '
              'spending power and helped swell the middle classes and the '
              'corporate sector.  Indias government has given its backing to '
              'cheaper and more accessible air travel.',
  'token': 16709,
  'token_str': ' booming'},
 {'score': 0.12074678391218185,
  'sequence': 'Recently however, the Indian economy has boosted personal '
              'spending power and helped swell the middle classes and the '
              'corporate sector.  Indias government has given its backing to '
              'cheaper and more accessible air travel.',
  'token': 1362,
  'token_str': ' Indian'},
 {'score': 0.05374400317668915,
  'sequence': 'Recently however, the sluggish economy has boosted personal '
              'spending power and helped swell the middle classes and the '
              'corporate sector.  Indias government has given its backing to '
        

In [26]:
text = 'Recently however, the booming economy has boosted personal spending power and helped swell the middle classes and the <mask> sector.  India''s government has given its backing to cheaper and more accessible air travel.'

pprint(mlm(text))

[{'score': 0.1763671636581421,
  'sequence': 'Recently however, the booming economy has boosted personal '
              'spending power and helped swell the middle classes and the '
              'private sector.  Indias government has given its backing to '
              'cheaper and more accessible air travel.',
  'token': 940,
  'token_str': ' private'},
 {'score': 0.15450620651245117,
  'sequence': 'Recently however, the booming economy has boosted personal '
              'spending power and helped swell the middle classes and the '
              'tourism sector.  Indias government has given its backing to '
              'cheaper and more accessible air travel.',
  'token': 5724,
  'token_str': ' tourism'},
 {'score': 0.12135636806488037,
  'sequence': 'Recently however, the booming economy has boosted personal '
              'spending power and helped swell the middle classes and the '
              'informal sector.  Indias government has given its backing to '
              

In [ ]:
# Exercise: Write a function that automatically masks and replaces words
# in a whole document. You might choose which words to replace based on some
# statistic, e.g. TF-IDF.

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform([doc])


In [38]:
words_set = vectorizer.get_feature_names_out()

In [34]:
tf_idf_array = X.toarray()

In [40]:
df_tf_idf = pd.DataFrame(tf_idf_array, columns = words_set)

df_tf_idf

,13,25,30,50,50m,60,8bn,931m,a320,abroad,...,travel,travellers,tuesday,uk,underserved,vijay,wants,won,year,years
0,0.0635,0.0635,0.0635,0.0635,0.0635,0.0635,0.127,0.0635,0.0635,0.0635,...,0.0635,0.0635,0.0635,0.0635,0.0635,0.0635,0.0635,0.0635,0.0635,0.0635


In [51]:
textNew = doc.replace("travel", "\<mask>")
pprint(mlm(textNew))

[[{'score': 0.2793717086315155,
   'sequence': "<s>India's Deccan seals $1.8bn deal\n"
               '\n'
               'Air Deccan has ordered 30 Airbus A320 planes in a $1.8bn '
               "(£931m) deal as India's first low-cost airline expands in the "
               'fast-growing domestic market.\n'
               '\n'
               'Air Deccan was set up last year and wants to lure \\ anglers '
               'away from the railway network and pricier rivals. The '
               'potential of the Indian market has attracted attention at home '
               'and abroad. Beer magnate Vijay Mallya recently set up '
               'Kingfisher Airlines, while UK entrepreneur Richard Branson has '
               'said he is keen to start a local operation.\n'
               '\n'
               'The country has a population of more than a billion people and '
               'many observers feel that it is underserved by airlines. '
               'Recently however, the booming 